In [ ]:
# I would reo

import json
from tqdm import tqdm
from PIL import Image
import torch
import requests
from io import BytesIO
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
from transformers import AutoProcessor, AutoModelForImageTextToText

In [ ]:
# %%capture
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install qwen-vl-utils
!pip install flash-attn --no-build-isolation # If this doesn't install you can ignore

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.9 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-msy263xw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-msy263xw
  Resolved https://github.com/huggingface/transformers.git to commit 98e8062df3cf82b367e8a243963e4afb0d9d3407
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10133629 sha256=d1090bd0770ef4621d158f83be928e9584c2659e2f8f052b6e500407a948a495
  Stored in directory: /tmp/pip-ephem-wheel-cache-y4rk8l4g/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existi

In [ ]:
from transformers import BitsAndBytesConfig, LlavaOnevisionForConditionalGeneration, LlavaOnevisionProcessor
from transformers import AutoProcessor, AutoModelForImageTextToText
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# model = LlavaOnevisionForConditionalGeneration.from_pretrained("llava-hf/llava-onevision-qwen2-0.5b-ov-hf", torch_dtype="float16", device_map='auto')
# processor = LlavaOnevisionProcessor.from_pretrained("llava-hf/llava-onevision-qwen2-0.5b-ov-hf")
# processor.tokenizer.padding_side = "left"


# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")
# model = AutoModelForImageTextToText.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
# )

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2",
    device_map="cpu",
)

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def load_image(image_file):
    image = None
    try:
      if image_file.startswith('http') or image_file.startswith('https'):
          response = requests.get(image_file)
          image = Image.open(BytesIO(response.content)).convert('RGB')
      else:
          image = Image.open(image_file).convert('RGB')
    except:
      print("There was an error reading the image file", image_file)
    return image

def generate_llama_onevision(model, processor, image, text):
  conversation = [
      {
          "role": "user",
          "content": [
              {"type": "text", "text": text},
              {"type": "image"}
          ]
      }
  ]

  prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
  inputs = processor(images=[image], text=[prompt], padding=True, return_tensors="pt").to(model.device, torch.float16)
  generate_kwargs = {"max_new_tokens": 50, "do_sample": True, "top_p": 0.9}
  output = model.generate(**inputs, **generate_kwargs)
  generated_text = processor.batch_decode(output, skip_special_tokens=True)
  response = generated_text[0].split("assistant\n")[1]
  return response

def generate_qwenvl(model, processor, image_path, text):
  conversation = [
      {
          "role": "user",
          "content": [
              {
                  "type": "image",
                  "image": f"{image_path}",
              },
              {"type": "text", "text": text},
          ],
      }
  ]

  text = processor.apply_chat_template(
      conversation, tokenize=False, add_generation_prompt=True
  )
  image_inputs, video_inputs = process_vision_info(conversation)
  inputs = processor(
      text=[text],
      images=image_inputs,
      videos=video_inputs,
      padding=True,
      return_tensors="pt",
  )
  inputs = inputs.to(model.device)

  # Inference: Generation of the output
  generated_ids = model.generate(**inputs, max_new_tokens=128)
  generated_ids_trimmed = [
      out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
  ]
  output_text = processor.batch_decode(
      generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
  )
  return output_text

In [ ]:
generate_qwenvl(model, processor, "https://c4.staticflickr.com/8/7211/7206072054_c53d53b97d_o.jpg", "What is happening in this image?")

In [ ]:
def eval_model(json_file, output):

    # Load input JSON data
    with open(json_file, 'r') as f:
        json_data = json.load(f)

    logs = []
    iter_cnt = {1: 0, 2: 0, 3: 0}

    for idx, line in tqdm(enumerate(json_data)):
        image_src = line['image_src']
        image = load_image(image_src)
        if image == None:
          continue
        question = line['question']
        qs = question

        # Initial question generation
        initial_answer = generate(model, processor, image, question)
        outputs = initial_answer
        revision = None

        print('Initial output:', outputs)

        # Feedback and refinement loop
        for i in range(3):
            fqs = (
                "Generate the feedback given initial answer referring to question and image.\n"
                f"Question: {question}\nInitial answer: {outputs}"
            )
            feedback1 = generate(model, processor, image, fqs)
            print('Feedback:', feedback1)

            rqs = (
                "Adjust the initial response considering the feedback and image.\n"
                f"Question: {question}\nInitial answer: {outputs}\nFeedback: {feedback1}"
            )
            revision = generate(model, processor, image, rqs)
            print("Revision:", revision)

            comparison_prompt = (
                f"{question}\nA. {outputs}\nB. {revision}\n"
                "Answer with the option's letter from the given choices directly."
            )
            comparison_result = generate(model, processor, image, comparison_prompt)

            print('Answer comparison:', comparison_result)

            if 'b' in comparison_result.lower():
                outputs = revision
            elif 'a' in comparison_result.lower():
                break

        iter_cnt[i + 1] += 1

        # Logging results
        logs.append({
            'question': question,
            'gold answer': line['gt_answer'],
            'initial answer': initial_answer,
            'feedback': feedback1,
            'revision': revision,
        })

        line['model_answer'] = outputs
        print('---------------------------------')
        print('question: ', question)
        print('answer: ', line['gt_answer'])
        print('initial pred: ', initial_answer)
        print('pred: ', outputs)
        print('---------------------------------')

    with open(output, 'w') as f:
        json.dump(json_data, f, indent="\t")

    return logs

In [ ]:
eval_model("response_template.json", "output.json")